In [80]:
from datasets import load_dataset
from transformers import BloomTokenizerFast, BloomForTokenClassification, DataCollatorForTokenClassification, AutoModelForTokenClassification, TrainingArguments, Trainer
from datasets import load_dataset
import torch
import os
!pip install accelerate -U


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [90]:
dataset

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1436
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 169
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 85
    })
})

In [81]:
dataset = load_dataset("Impl.py", name="Kayan Dataset")

In [82]:
model_name = "bloom-560m"
tokenizer = BloomTokenizerFast.from_pretrained(f"bigscience/{model_name}", add_prefix_space=True)
model = BloomForTokenClassification.from_pretrained(f"bigscience/{model_name}")

Some weights of BloomForTokenClassification were not initialized from the model checkpoint at bigscience/bloom-560m and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [83]:
def tokenizeInputs(inputs):
    tokenized_inputs = tokenizer(inputs["tokens"], truncation=True, max_length=512, is_split_into_words=True)
    word_ids = tokenized_inputs.word_ids()
    ner_tags = inputs["ner_tags"]
    labels = [ner_tags[word_id] for word_id in word_ids]
    tokenized_inputs["labels"] = labels

    return tokenized_inputs

In [84]:
tokenized_datasets = dataset.map(tokenizeInputs)

Map:   0%|          | 0/1436 [00:00<?, ? examples/s]

Map:   0%|          | 0/169 [00:00<?, ? examples/s]

Map:   0%|          | 0/85 [00:00<?, ? examples/s]

In [85]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [86]:
model = AutoModelForTokenClassification.from_pretrained(f"bigscience/{model_name}", num_labels=12)

Some weights of BloomForTokenClassification were not initialized from the model checkpoint at bigscience/bloom-560m and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [87]:
print("Parameters:", model.num_parameters())
print("Expected Input Dict:", model.main_input_name )

sample = tokenized_datasets["train"][0]
sample["input_ids"] = torch.Tensor(sample["input_ids"])
flops_est = model.floating_point_ops(input_dict = sample, exclude_embeddings = False)

print("FLOPS needed per Training Sample:", flops_est )

Parameters: 559226892
Expected Input Dict: input_ids
FLOPS needed per Training Sample: 1201219364016


In [89]:
training_args = TrainingArguments(
    output_dir="./results",
    save_strategy= "epoch", # Disabled for runtime evaluation
    evaluation_strategy="steps", #"steps", # Disabled for runtime evaluation
    eval_steps = 500,
    learning_rate=2e-5,
    per_device_train_batch_size=12,
    per_device_eval_batch_size=12,
    num_train_epochs=2,
    weight_decay=0.01
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["valid"],
    tokenizer=tokenizer,
    data_collator=data_collator
)

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
trainer.train()